In [27]:
import pandas as pd 
import numpy as np 
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import torch 
import torch.nn as nn
from torch_geometric.nn import MessagePassing
import torch_geometric as pyg
from torch_geometric.utils.negative_sampling import structured_negative_sampling
from torch_geometric.transforms import gcn_norm

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
print("Unique User  IDs: ", ratings['userId'].nunique())
print("Unique Movie IDs: ", ratings['movieId'].nunique())
print("Max User ID: ", ratings['userId'].max())
print("Max Movie ID: ", ratings['movieId'].max())

Unique User  IDs:  610
Unique Movie IDs:  9724
Max User ID:  610
Max Movie ID:  193609


In [4]:
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

ratings['userId']  = user_encoder.fit_transform(ratings['userId'])
ratings['movieId'] = movie_encoder.fit_transform(ratings['movieId'])

In [5]:
print("Unique User  IDs: ", ratings['userId'].nunique())
print("Unique Movie IDs: ", ratings['movieId'].nunique())
print("Max User ID: ", ratings['userId'].max())
print("Max Movie ID: ", ratings['movieId'].max())

num_users  = ratings['userId'].nunique()
num_movies = ratings['movieId'].nunique()

Unique User  IDs:  610
Unique Movie IDs:  9724
Max User ID:  609
Max Movie ID:  9723


In [6]:
def create_interaction_edges(userids, movieids, ratings_, threshold=3.5):
    ''' Interaction edges in COO format.'''
    mask = ratings_ > threshold
    edges = np.stack([userids[mask], movieids[mask]])
    return torch.LongTensor(edges)


In [7]:
def create_adj_matrix(int_edges, num_users, num_movies):

    n = num_users + num_movies
    adj = torch.zeros(n,n)

    r_mat = torch.sparse_coo_tensor(int_edges, torch.ones(int_edges.shape[1]), size=(num_users, num_movies)).to_dense()
    adj[:num_users, num_users:] = r_mat.clone()
    adj[num_users:, :num_users] = r_mat.T.clone()

    adj_coo = adj.to_sparse_coo()
    adj_coo = adj_coo.indices()

    return adj_coo

In [ ]:
def  adj_to_r_mat(adj, num_users, num_movies):
    adj_dense = torch.sparse_coo_tensor(adj, torch.ones(adj.shape[1]), size=(num_users, num_movies)).to_dense()
    r_mat = adj_dense[:num_users, num_users:]
    r_coo = r_mat.to_sparse_coo()
    return r_coo.indices()

In [8]:
int_edges = create_interaction_edges(ratings['userId'], ratings['movieId'], ratings['rating'])

In [9]:
int_edges.shape 

torch.Size([2, 48580])

In [10]:
indices = torch.arange(0, int_edges.shape[1], dtype=torch.long)

train_idx, test_idx = train_test_split(indices, test_size=0.2)
val_idx, test_idx   = train_test_split(test_idx, test_size=0.5)

train_edges = int_edges[:, train_idx]
val_edges   = int_edges[:, val_idx]
test_edges  = int_edges[:, test_idx]

print("Train edges: ", train_edges.shape)
print("Val   edges: ", val_edges.shape)
print("Test  edges: ", test_edges.shape)

Train edges:  torch.Size([2, 38864])
Val   edges:  torch.Size([2, 4858])
Test  edges:  torch.Size([2, 4858])


In [11]:
train_adj = create_adj_matrix(train_edges, num_users, num_movies)
val_adj   = create_adj_matrix(val_edges, num_users, num_movies)
test_adj  = create_adj_matrix(test_edges, num_users, num_movies)


In [20]:
## edges = (i, j, k) (i, j) positive edge (i, k) negative edge
def sample_mini_batch(edge_index, batch_size):

    '''
    Args:
        edge_index: edge_index of the user-item interaction matrix
    
    Return:
        structured_negative_sampling return (i,j,k) where
            (i,j) positive edge
            (i,k) negative edge
    '''
    edges = structured_negative_sampling(edge_index)
    edges = torch.stack(edges, dim=0)
    random_idx = random.choices(list(range(edges[0].shape[0])), k=batch_size)
    batch = edges[:, random_idx]
    user_ids, pos_items, neg_items = batch[0], batch[1], batch[2]
    return user_ids, pos_items, neg_items


In [ ]:
class LightGCNConv(MessagePassing):
    ''' How message passing base class works?
        https://zqfang.github.io/2021-08-07-graph-pyg/#messagepassing-in-pytorch-geometric 
        j : Source index
        i : Target index
    '''
    def __init__(self):
        super().__init__(aggr='add')

    def forward(self, x, edge_index):
        norm = gcn_norm(edge_index)
        out = self.propagate(edge_index, x=x, norm=norm)
        return out

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
    

class LightGCN(torch.nn.Module):
    def __init__(self, num_users, num_items, emb_dim = 64, num_layers=4):
        super().__init__()
        
        self.userEmb = nn.Embedding(num_users, emb_dim)
        self.itemEmb = nn.Embedding(num_items, emb_dim)

        self.num_layers = num_layers

        self.convs = nn.ModuleList()
        for i in range(self.num_layers):
            self.convs.append(LightGCNConv())

    def forward(self, edge_index):
        emb = torch.cat([self.userEmb, self.itemEmb], dim=0) 
        emb_lst = [emb]

        for lightgcn in self.convs:
            emb = lightgcn(emb, edge_index)
            emb_lst.append(emb_lst)
        
        out_emb = torch.stack(emb_lst, dim=1)   
        out_emb = torch.mean(out_emb, dim=1)

        # Split user and item embeddings
        users_emb_final, items_emb_final = torch.split(out_emb, [self.num_users, self.num_items]) 

        return users_emb_final, self.userEmb.weight, items_emb_final, self.itemEmb.weight

In [ ]:
def bpr_loss(user_emb, user_emb_0, item_emb, item_emb_0, edge_index_r, batch_size = 128, lambda_= 1e-4):

    user_ids, pos_items, neg_items = sample_mini_batch(edge_index_r, batch_size=batch_size)

    user_emb_sub = user_emb[user_ids]
    pos_item_emb = item_emb[pos_items]
    neg_item_emb = item_emb[neg_items]

    pos_scores = torch.diag(user_emb_sub @ pos_item_emb.T)
    neg_scores = torch.diag(user_emb_sub @ neg_item_emb.T)

    reg_loss = lambda_*(
        user_emb_0[user_ids].norm(2).pow(2) +
        item_emb_0[pos_item_emb].norm(2).pow(2) +
        item_emb_0[neg_item_emb].norm(2).pow(2) # L2 loss
    )

    loss = -torch.mean(torch.nn.functional.softplus(pos_scores - neg_scores)) + reg_loss
    return loss
    